In [1]:
import numpy as np
import torch
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class MyANN(nn.Module):
    def __init__(self):
        super(MyANN, self).__init__(inputs, out1, out2, class_num)
        self.fc1 = nn.Sequential(
            nn.Linear(inputs, out1),
            nn.BatchNorm1d(out1),   # 添加批标准化
            nn.ReLU(inplace=True)
        )
        self.fc2 = nn.Sequential(
            nn.linear(out1, out2),
            nn.BatchNorm1d(out2),
            nn.ReLU(inplace=True)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(out2, class_num)
        )
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x